# Create a Graph from a text

This notebook demonstrates how to extract graph from any text using the graph maker

Steps:
- Define an Ontology
- Load a list of example text chunks. We will use the Lord of the Rings summary from this wikipedia page. 
- Create Graph using an Open source model using Groq APIs. 
- Save the graph to Neo4j db
- Visualise



Loading the graph maker functions ->

In [3]:
from graph_maker import GraphMaker, Ontology, GroqClient
from graph_maker import Document


# Define the Ontology. 

The ontology is a pydantic model with the following schema. 

```python
class Ontology(BaseModel):
    label: List[Union[str, Dict]]
    relationships: List[str]
```

Here is the ontology we will use for the LOTR summaries ->


In [4]:
ontology = Ontology(
    labels=[
        {"Research Paper" : "a research document document" },
        {"Machine Learning Task" : "The specific problem the model addresses"},
        {"Model Architecture" : '''The overall architecture of the ML model which includes 
        Layers, that is Different layers within the model (e.g., input, convolutional, recurrent, output).
        Activation Functions, that is Functions like ReLU, Sigmoid, etc.
        Hyperparameters, like Learning rate, number of epochs, batch size, etc.
        Optimizers, that is Algorithms to update the weight parameter (e.g., SGD, Adam).'''},
        {"Data_structures" : "The type of structure of the data (for example, arrays, Pandas DataFrames, Matrices, tensors etc.)"},
        {"Frameworks for Data Manipulation" : " different libraries for data manipulation like pytorch, tensorflow etc."},
        {"Dataset": "Data used for training and testing. This includes Data Preprocessing, which are techniques used to prepare data (normalization, augmentation)"},
        {"Evaluation Metric" : "Metrics used to assess performance (accuracy, F1 score, etc.)"},
        {"Results": "Outcomes of experiments, including statistical measures."},
        {"Experimental Setup" : '''Details about how experiments were structured. This includes but is not limited to
        Hardware: Specifications of the computing resources used.
        Software: Software and tools, versions used.'''},
        {"subtask" : "any task which is part of the whole process of achieveing the bigger goal"},
        {"method" : "the methodology used to achieve a subtask. This includes the logical sequence (which can be potentially converted to code) of steps, libraries and tools used"},
        {"tool" : "The standard or custom tools used in a method"}, 
        {"Miscellaneous": "Any important concept can not be categorised with any other given label"},
    ],
    relationships=[
        "Proposes : Links Research Paper to Model Architecture.",
        "Addresses : Connects Model Architecture to Machine Learning Task.",
        "Includes Layer : From Model Architecture to Layers.",
        "Uses Activation : From Layers to Activation Functions.",
        "Sets Hyperparameter : From Model Architecture to Hyperparameters.",
        "Utilizes Optimizer : From Model Architecture to Optimizers.",
        "Applies Preprocessing : From Dataset to Data Preprocessing methods.",
        "Uses : Links Model Architecture to Dataset.",
        "Evaluates With : Connects Model Architecture to Evaluation Metric.",
        "Reports Results : From Research Paper to Results.",
        "Conducted On : From Experimental Setup to Hardware and Software.", 
        "method for subtask : from method to subtask",
        "tool for method : from tool to method",
        "Misc : Miscellaneous relationship between any pair of Entities",
]
)


List of text chunks. These are the summaries of the LOTR books from the Wikipedia page. 

In [1]:
# # from lotr_wikipedia_summary import lord_of_the_rings_wikipedia_summary as example_text_list
# # len(example_text_list)

# example_text_list = ['''In this section we first describe the core layers that Mo-
# bileNet is built on which are depthwise separable filters.
# We then describe the MobileNet network structure and con-
# clude with descriptions of the two model shrinking hyper-
# parameters width multiplier and resolution multiplier.''',

# '''The MobileNet model is based on depthwise separable
# convolutions which is a form of factorized convolutions
# which factorize a standard convolution into a depthwise
# convolution and a 1 × 1 convolution called a pointwise con-
# volution. For MobileNets the depthwise convolution ap-
# plies a single filter to each input channel. The pointwise
# convolution then applies a 1 × 1 convolution to combine the
# outputs the depthwise convolution. A standard convolution
# both filters and combines inputs into a new set of outputs
# in one step. The depthwise separable convolution splits this
# into two layers, a separate layer for filtering and a separate
# layer for combining. This factorization has the effect of
# drastically reducing computation and model size. Figure 2
# shows how a standard convolution 2(a) is factorized into a
# depthwise convolution 2(b) and a 1 × 1 pointwise convolu-
# tion 2(c).''',

# '''A standard convolutional layer takes as input a DF ×DF × M feature map F and produces a DF × DF × N
# feature map G where DF is the spatial width and height
# of a square input feature map1, M is the number of input
# channels (input depth), DG is the spatial width and height of
# a square output feature map and N is the number of output
# channel (output depth).
# The standard convolutional layer is parameterized by
# convolution kernel K of size DK ×DK ×M ×N where DK
# is the spatial dimension of the kernel assumed to be square
# and M is number of input channels and N is the number of
# output channels as defined previously.
# The output feature map for standard convolution assum-
# ing stride one and padding is computed as:
# Gk,l,n = ∑
# i,j,m
# Ki,j,m,n · Fk+i−1,l+j−1,m (1)
# Standard convolutions have the computational cost of:
# DK · DK · M · N · DF · DF
# where the computational cost depends multiplicatively on
# the number of input channels M , the number of output
# channels N the kernel size Dk × Dk and the feature map
# size DF × DF . MobileNet models address each of these
# terms and their interactions. First it uses depthwise separa-
# ble convolutions to break the interaction between the num-
# ber of output channels and the size of the kernel.
# The standard convolution operation has the effect of fil-
# tering features based on the convolutional kernels and com-
# bining features in order to produce a new representation.
# The filtering and combination steps can be split into two
# steps via the use of factorized convolutions called depthwise 
# separable convolutions for substantial reduction in compu-
# tational cost.
# Depthwise separable convolution are made up of two
# layers: depthwise convolutions and pointwise convolutions.
# We use depthwise convolutions to apply a single filter per
# each input channel (input depth). Pointwise convolution, a
# simple 1×1 convolution, is then used to create a linear com-
# bination of the output of the depthwise layer. MobileNets
# use both batchnorm and ReLU nonlinearities for both lay-
# ers.
# Depthwise convolution with one filter per input channel
# (input depth) can be written as:
# ˆGk,l,m = ∑
# i,j
# ˆKi,j,m · Fk+i−1,l+j−1,m (3)
# where ˆK is the depthwise convolutional kernel of size
# DK × DK × M where the mth filter in ˆK is applied to
# the mth channel in F to produce the mth channel of the
# filtered output feature map ˆG.
# Depthwise convolution has a computational cost of:
# DK · DK · M · DF · DF (4)
# Depthwise convolution is extremely efficient relative to
# standard convolution. However it only filters input chan-
# nels, it does not combine them to create new features. So
# an additional layer that computes a linear combination of
# the output of depthwise convolution via 1 × 1 convolution
# is needed in order to generate these new features.
# The combination of depthwise convolution and 1 × 1
# (pointwise) convolution is called depthwise separable con-
# volution which was originally introduced in [26].
# Depthwise separable convolutions cost:
# DK · DK · M · DF · DF + M · N · DF · DF (5)
# which is the sum of the depthwise and 1 × 1 pointwise con-
# volutions.
# By expressing convolution as a two step process of filter-
# ing and combining we get a reduction in computation of:
# DK · DK · M · DF · DF + M · N · DF · DF
# DK · DK · M · N · DF · DF
# = 1
# N + 1
# D2
# K
# MobileNet uses 3 × 3 depthwise separable convolutions
# which uses between 8 to 9 times less computation than stan-
# dard convolutions at only a small reduction in accuracy as
# seen in Section 4.
# Additional factorization in spatial dimension such as in
# [16, 31] does not save much additional computation as very
# little computation is spent in depthwise convolutions.''',

# '''The MobileNet structure is built on depthwise separable
# convolutions as mentioned in the previous section except for
# the first layer which is a full convolution. By defining the
# network in such simple terms we are able to easily explore
# network topologies to find a good network. The MobileNet
# architecture is defined in Table 1. All layers are followed by
# a batchnorm [13] and ReLU nonlinearity with the exception
# of the final fully connected layer which has no nonlinearity
# and feeds into a softmax layer for classification. Figure 3
# contrasts a layer with regular convolutions, batchnorm and
# ReLU nonlinearity to the factorized layer with depthwise
# convolution, 1 × 1 pointwise convolution as well as batch-
# norm and ReLU after each convolutional layer. Down sam-
# pling is handled with strided convolution in the depthwise
# convolutions as well as in the first layer. A final average
# pooling reduces the spatial resolution to 1 before the fully
# connected layer. Counting depthwise and pointwise convo-
# lutions as separate layers, MobileNet has 28 layers.
# It is not enough to simply define networks in terms of a
# small number of Mult-Adds. It is also important to make
# sure these operations can be efficiently implementable. For 
# instance unstructured sparse matrix operations are not typ-
# ically faster than dense matrix operations until a very high
# level of sparsity. Our model structure puts nearly all of the
# computation into dense 1 × 1 convolutions. This can be im-
# plemented with highly optimized general matrix multiply
# (GEMM) functions. Often convolutions are implemented
# by a GEMM but require an initial reordering in memory
# called im2col in order to map it to a GEMM. For instance,
# this approach is used in the popular Caffe package [15].
# 1 × 1 convolutions do not require this reordering in memory
# and can be implemented directly with GEMM which is one
# of the most optimized numerical linear algebra algorithms.
# MobileNet spends 95% of it’s computation time in 1 × 1
# convolutions which also has 75% of the parameters as can
# be seen in Table 2. Nearly all of the additional parameters
# are in the fully connected layer.
# MobileNet models were trained in TensorFlow [1] us-
# ing RMSprop [33] with asynchronous gradient descent sim-
# ilar to Inception V3 [31]. However, contrary to training
# large models we use less regularization and data augmen-
# tation techniques because small models have less trouble
# with overfitting. When training MobileNets we do not use
# side heads or label smoothing and additionally reduce the
# amount image of distortions by limiting the size of small
# crops that are used in large Inception training [31]. Addi-
# tionally, we found that it was important to put very little or
# no weight decay (l2 regularization) on the depthwise filters
# since their are so few parameters in them. For the ImageNet
# benchmarks in the next section all models were trained with
# same training parameters regardless of the size of the model''',

# '''Although the base MobileNet architecture is already
# small and low latency, many times a specific use case or
# application may require the model to be smaller and faster.
# In order to construct these smaller and less computationally
# expensive models we introduce a very simple parameter α
# called width multiplier. The role of the width multiplier α is
# to thin a network uniformly at each layer. For a given layer and width multiplier α, the number of input channels M be-
# comes αM and the number of output channels N becomes
# αN .
# The computational cost of a depthwise separable convo-
# lution with width multiplier α is:
# DK · DK · αM · DF · DF + αM · αN · DF · DF (6)
# where α ∈ (0, 1] with typical settings of 1, 0.75, 0.5 and
# 0.25. α = 1 is the baseline MobileNet and α < 1 are
# reduced MobileNets. Width multiplier has the effect of re-
# ducing computational cost and the number of parameters
# quadratically by roughly α2. Width multiplier can be ap-
# plied to any model structure to define a new smaller model
# with a reasonable accuracy, latency and size trade off. It
# is used to define a new reduced structure that needs to be
# trained from scratch''',

# '''The second hyper-parameter to reduce the computational
# cost of a neural network is a resolution multiplier ρ. We apply this to the input image and the internal representation of
# every layer is subsequently reduced by the same multiplier.
# In practice we implicitly set ρ by setting the input resolu-
# tion.
# We can now express the computational cost for the core
# layers of our network as depthwise separable convolutions
# with width multiplier α and resolution multiplier ρ:
# DK · DK · αM · ρDF · ρDF + αM · αN · ρDF · ρDF (7)
# where ρ ∈ (0, 1] which is typically set implicitly so that
# the input resolution of the network is 224, 192, 160 or 128.
# ρ = 1 is the baseline MobileNet and ρ < 1 are reduced
# computation MobileNets. Resolution multiplier has the ef-
# fect of reducing computational cost by ρ2.
# As an example we can look at a typical layer in Mo-
# bileNet and see how depthwise separable convolutions,
# width multiplier and resolution multiplier reduce the cost
# and parameters. Table 3 shows the computation and number
# of parameters for a layer as architecture shrinking methods
# are sequentially applied to the layer. The first row shows
# the Mult-Adds and parameters for a full convolutional layer
# with an input feature map of size 14 × 14 × 512 with a ker-
# nel K of size 3 × 3 × 512 × 512. We will look in detail
# in the next section at the trade offs between resources and
# accuracy.''']





# '''In this work, we introduce MUSICGEN, a simple and controllable music generation model, which is able to generate high-quality music given textual description. We propose a general framework for
# modeling multiple parallel streams of acoustic tokens, which serves as a generalization of previous
# studies (see Figure 1). We show how this framework allows to extend generation to stereo audio at
# no extra computational cost. To improve controllability of the generated samples, we additionally
# introduce unsupervised melody conditioning, which allows the model to generate music that matches
# a given harmonic and melodic structure. We conduct an extensive evaluation of MUSICGEN and
# show the proposed method is superior to the evaluated baselines by a large margin, with a subjective
# rating of 84.8 out of 100 for MUSICGEN against 80.5 for the best baseline. We additionally provide
# an ablation study which sheds light on the importance of each of the components on the overall model
# performance. Lastly, human evaluation suggests that MUSICGEN yields high quality samples which
# are better melodically aligned with a given harmonic structure, while adhering to a textual description''',

# '''Our contribution: (i) We introduce a simple and efficient model to generate high quality music at
# 32 kHz. We show that MUSICGEN can generate consistent music with a single-stage language model
# through an efficient codebook interleaving strategy. (ii) We present a single model to perform both
# text and melody-conditioned generation and demonstrate that the generated audio is coherent with
# the provided melody and faithful to the text conditioning information. (iii) We provide extensive
# objective and human evaluations on the key design choices behind our method''',

# '''2 Method
# MUSICGEN consists in an autoregressive transformer-based decoder [Vaswani et al., 2017], condi-
# tioned on a text or melody representation. The (language) model is over the quantized units from
# an EnCodec [Défossez et al., 2022] audio tokenizer, which provides high fidelity reconstruction
# from a low frame rate discrete representation. Compression models such as [Défossez et al., 2022,
# Zeghidour et al., 2021] employ Residual Vector Quantization (RVQ) which results in several parallel
# streams. Under this setting, each stream is comprised of discrete tokens originating from different
# learned codebooks. Prior work, proposed several modeling strategies to handle this issue [Kharitonov
# et al., 2022, Agostinelli et al., 2023, Wang et al., 2023]. In this work, we introduce a novel modeling
# framework, which generalizes to various codebook interleaving patterns, and we explore several
# 2
# variants. Through patterns, we can leverage the internal structure of the quantized audio tokens.
# Finally, MUSICGEN supports conditional generation based on either text or melody''',

# '''2.1 Audio tokenization
# We use EnCodec [Défossez et al., 2022], a convolutional auto-encoder with a latent space quantized
# using Residual Vector Quantization (RVQ) [Zeghidour et al., 2021], and an adversarial reconstruction
# loss. Given a reference audio random variable X ∈ Rd·fs with d the audio duration and fs the sample
# rate, EnCodec encodes it into a continuous tensor with a frame rate fr ≪ fs. This representation
# is then quantized into Q ∈ {1, . . . , M }d·fr ×K , with K being the number of codebooks used in RVQ
# and M being the codebook size. Notice, after quantization we are left with K parallel discrete tokens
# sequences, each of length T = d · fr , representing the audio sample. In RVQ, each quantizer encodes
# the quantization error left by the previous quantizer, thus quantized values for different codebooks
# are in general not independent, and the first codebook is the most important one.''',

# '''2.2 Codebook interleaving patterns (see Figure 1)
# Exact flattened autoregressive decomposition. An autoregressive model requires a discrete random
# sequence U ∈ {1, . . . , M }S with S the sequence length. By convention, we will take U0 = 0,
# a deterministic special token indicating the beginning of the sequence. We can then model the
# distribution
# ∀t > 0, pt (Ut−1, . . . , U0) ≜ P [Ut|Ut−1, . . . , U0] . (1)
# Let us build a second sequence of random variables ˜U using the auto-regressive density p, e.g. we
# define recursively ˜U0 = 0, and for all t > 0,
# ∀t > 0, P
# h ˜Ut| ˜Ut−1 . . . , ˜U0
# i
# = pt
#  ˜Ut−1, . . . , ˜U0
# 
# . (2)
# Then, we immediately have that U and ˜U follow the same distribution. This means that if we can fit a
# perfect estimate ˆp of p with a deep learning model, then we can fit exactly the distribution of U .
# As stated before, the main issue with the representation Q we obtained from the EnCodec model is
# that there are K codebooks for each time step. One solution would be to flatten out Q, thus taking
# S = d · fr · K, e.g. first predicting the first codebook of the first time step, then the second codebook
# of the first time step, etc. Then, using eq. (1) and eq. (2) , we could theoretically fit an exact model
# of the distribution of Q. The downside however is the increased complexity, with part of the gain
# coming from the lowest sample rate fr being lost.
# More than one possible flattening exists, and not all the ˆpt functions need to be estimated through a
# single model. For instance, MusicLM [Agostinelli et al., 2023] uses two models, one modeling the
# flattened first K/2 codebooks, and a second one the other K/2 flattened codebooks, conditioned on
# the decision of the first model. In that case, the number of autoregressive steps is still dfr · K''',

# '''nexact autoregressive decomposition. Another possibility is to consider an autoregressive de-
# composition, where some codebooks are predicted in parallel. For instance, let us define another
# sequence with V0 = 0 and for all t ∈ {1, . . . , T }, k ∈ {1, . . . , K}, Vt,k = Qt,k. When dropping the
# codebook index k, e.g. Vt, we mean the concatenation of all the codebooks at time t.
# pt,k (Vt−1, . . . , V0) ≜ P [Vt,k|Vt−1, ·, . . . , V0] . (3)
# Let’s define again recursively ˜V0 = 0 and for all t > 0,
# ∀t > 0, ∀k, P
# h ˜Vt,k
# i
# = pt,k
#  ˜Vt−1, . . . , ˜V0
# 
# . (4)
# Unlike in (2), we no longer have in the general case that ˜V follows the same distribution as V ,
# even assuming we have access to the exact distribution pt,k. In fact, we would only have a proper
# generative model if for all t, (Vt,k)k are independent conditionally on Vt−1, . . . , V0. As t increases,
# the errors will compound and the two distributions can grow further apart. Such a decomposition
# is inexact, but allows to keep the original frame rate which can considerably speed up training and
# inference, especially for long sequences.''',

# '''Arbitrary codebook interleaving patterns. In order to experiment with various such decompositions,
# and measure exactly the impact of using an inexact decomposition, we introduce codebook interleav-
# ing patterns. Let us consider Ω = {(t, k) : {1, . . . , d · fr }, k ∈ {1, . . . , K}} be the set of all pairs of
# time steps and codebook indexes. A codebook pattern is a sequence P = (P0, P1, P2, . . . , PS ), with
# P0 = ∅, and for all 0 < s ≤ S, Ps ⊂ Ω, such that P is partition of Ω. We model Q by predicting in
# parallel all the positions in Ps, conditionally on all the positions in P0, P1, . . . , Ps−1. Pragmatically,
# we restrict ourselves to patterns where each codebook index appears at most once in any of the Ps.
# We can now easily define a number of decompositions, for instance the “parallel” pattern given by
# Ps = {(s, k) : k ∈ {1, . . . , K}}. (5)
# It is also possible to introduce a “delay” between the codebooks, as in Kharitonov et al. [2022], e.g.,
# Ps = {(s − k + 1, k) : k ∈ {1, . . . , K}, s − k ≥ 0}. (6)
# Through empirical evaluations, we show the benefits and drawbacks of various codebook patterns,
# shedding light on the importance of exact modeling of the parallel codebook sequences.''',

# ''' 2.3 Model conditioning
# Text conditioning. Given a textual description matching the input audio X, we compute a condi-
# tioning tensor C ∈ RTC ×D with D being the inner dimension used in the autoregressive model.
# Generally, there are three main approaches for representing text for conditional audio generation.
# Kreuk et al. [2022] proposed using a pretrained text encoder, specifically T5 [Raffel et al., 2020].
# Chung et al. [2022] show that using instruct-based language models provide superior performance.
# Lastly, Agostinelli et al. [2023], Liu et al. [2023], Huang et al. [2023a], Sheffer and Adi [2023]
# claimed that joint text-audio representation, such as CLAP [Wu* et al., 2023], provides better-quality
# generations. We experiment with all of the above, respectively: T5 encoder, FLAN-T5, and CLAP.
# Melody conditioning. While text is the prominent approach in conditional generative models
# nowadays, a more natural approach for music is conditioning on a melodic structure from another
# audio track or even whistling or humming. Such an approach also allows for an iterative refinement
# of the model’s output. To support that, we experiment with controlling the melodic structure via
# jointly conditioning on the input’s chromagram and text description. In preliminary experiments, we
# observed that conditioning on the raw chromagram often led to reconstructing the original sample,
# resulting in overfitting. To reduce it, we introduce an information bottleneck by choosing the dominant
# time-frequency bin in each time step. While a similar capability was shown in Agostinelli et al.
# [2023], the authors used supervised proprietary data, which is tedious and costly to collect. In this
# work, we take an unsupervised approach, eliminating the requirement for supervised data.''',

# '''2.4 Model architecture
# Codebook projection and positional embedding. Given a codebook pattern, only some codebooks
# are present at each pattern step Ps. We retrieve from Q the values corresponding to the indices in Ps.
# As noted in Section 2.2, each codebook is present at most once in Ps or not at all. If it is present,
# we use a learned embedding table with N entries and dimension D to represent the associated value
# from Q. Otherwise, we use a special token indicating its absence. We sum the contribution from each
# codebook after this transformation. As P0 = ∅, the first input is always the sum of all the special
# tokens. Finally, we sum a sinusoidal embedding to encode the current step s [Vaswani et al., 2017].
# Transformer decoder. The input is fed into a transformer with L layers and a dimension D. Each
# layer consists of a causal self-attention block. We then use a cross-attention block that is fed with the
# conditioning signal C. When using melody conditioning, we instead provide the conditioning tensor
# C as a prefix to the transformer input. The layer ends with a fully connected block consisting of a
# linear layer from D to 4·D channels, a ReLU, and a linear layer back to D channels. The attention and
# fully connected blocks are wrapped with a residual skip connection. Layer normalization [Ba et al.,
# 2016] is applied to each block before being summed with the residual skip connection (“pre-norm”).
# Logits prediction. The output from the transformer decoder at pattern step Ps is transformed into
# logits prediction for the values of Q taken at the indices given by Ps+1. Each codebook is present at
# most once in Ps+1. If a codebook is present, the logits prediction is obtained by applying a codebook
# specific linear layer from D channels to N ''',

# '''3 Experimental setup
# 3.1 Models and hyperparameters
# Audio tokenization model. We use a non-causal five layers EnCodec model for 32 kHz monophonic
# audio with a stride of 640, resulting in a frame rate of 50 Hz, and an initial hidden size of 64, doubling
# at each of the model’s five layers. The embeddings are quantized with a RVQ with four quantizers,
# each with a codebook size of 2048. We follow Défossez et al. [2022] to train the model on one-second
# audio segments cropped at random in the audio sequence.
# Transformer model. We train autoregressive transformer models at different sizes: 300M, 1.5B,
# 3.3B parameters. We use a memory efficient Flash attention [Dao et al., 2022] from the xFormers
# package [Lefaudeux et al., 2022] to improve both speed and memory usage with long sequences. We
# study the impact of the size of the model in Section 4. We use the 300M-parameter model for all of
# our ablations. We train on 30-second audio crops sampled at random from the full track. We train
# the models for 1M steps with the AdamW optimizer [Loshchilov and Hutter, 2017], a batch size of
# 192 examples, β1 = 0.9, β2 = 0.95, a decoupled weight decay of 0.1 and gradient clipping of 1.0.
# We further rely on D-Adaptation based automatic step-sizes [Defazio and Mishchenko, 2023] for the
# 300M model as it improves model convergence but showed no gain for the bigger models. We use
# a cosine learning rate schedule with a warmup of 4000 steps. Additionally, we use an exponential
# moving average with a decay of 0.99. We train the 300M, 1.5B and 3.3B parameter models, using
# respectively 32, 64 and 96 GPUs, with mixed precision. More specifically, we use float16 as bfloat16
# was leading to instabilities in our setup. Finally, for sampling, we employ top-k sampling [Fan et al.,
# 2018] with keeping the top 250 tokens and a temperature of 1.0.
# Text preprocessing. Kreuk et al. [2022] proposed a text normalization scheme, in which stop words
# are omitted and the remaining text is lemmatized. We denote this method by text-normalization.
# When considering musical datasets, additional annotations tags such as musical key, tempo, type
# of instruments, etc. are often available. We also experiment with concatenating such annotations
# to the text description. We denote this approach by condition-merging. Finally, we explored using
# word dropout as another text augmentation strategy. For the final models, we used condition-merging
# with a probability of 0.25. Upon merging, we apply a text description dropout with a probability of
# 0.5. We use a word dropout with a probability of 0.3 on the resulting text. A full comparison of the
# different text preprocessing strategies can be found in Appendix A.2.
# Codebook patterns and conditioning. We use the “delay” interleaving pattern from Section 2.2,
# This translates 30 seconds of audio into 1500 autoregressive steps. For text conditioning, we use
# the T5 [Raffel et al., 2020] text encoder, optionally with the addition of the melody conditioning
# presented in Section 2.3. We also experiment with FLAN-T5 [Chung et al., 2022], and CLAP [Wu*
# et al., 2023] and compare the performance of MUSICGEN using each of these text encoders in the
# Appendix A.2. For melody conditioning, we compute the chromagrams with a window size of 214
# and a hop size of 212. Using a large window prevents the model from recovering fine temporal
# details. We further quantize the chromagram by taking the argmax at each time step. We follow a
# similar approach to Kreuk et al. [2022] and implement classifier-free guidance when sampling from
# the model’s logits. Specifically, during training we drop the condition with a probability of 0.2 and
# during inference we use a guidance scale of 3.0''',

# '''3.2 Datasets
# Training datasets. We use 20K hours of licensed music to train MUSICGEN. Specifically, we rely on
# an internal dataset of 10K high-quality music tracks, and on the ShutterStock and Pond5 music data
# collections2 with respectively 25K and 365K instrument-only music tracks. All datasets consist of
# full-length music sampled at 32 kHz with metadata composed of a textual description and information
# such as the genre, BPM, and tags. We downmix the audio to mono unless stated otherwise.
# Evaluation datasets. For the main results and comparison with prior work, we evaluate the proposed
# method on the MusicCaps benchmark [Agostinelli et al., 2023]. MusicCaps is composed of 5.5K
# samples (ten-second long) prepared by expert musicians and a 1K subset balanced across genres. We
# report objective metrics on the unbalanced set, while we sample examples from the genre-balanced set for qualitative evaluations. 
# For melody evaluation and the ablation studies, we use samples from
# an in-domain held out evaluation set of 528 music tracks, with no artist overlap with the training set.''',

# '''Evaluation metrics. We evaluate the proposed method using objective and subjective metrics. \n
# For the objective methods, we use three metrics: the Fréchet Audio Distance (FAD), the Kullback-Leiber \n
# Divergence (KL) and the CLAP score (CLAP). We report the FAD [Kilgour et al., 2018] using the
# official implementation in Tensorflow with the VGGish model 5. A low FAD score indicates the
# generated audio is plausible. Following Kreuk et al. [2022], we use a state-of-the-art audio classifier
# trained for classification on AudioSet [Koutini et al., 2021] to compute the KL-divergence over the
# probabilities of the labels between the original and the generated music. The generated music is
# expected to share similar concepts with the reference music when the KL is low. Last, the CLAP
# score [Wu* et al., 2023, Huang et al., 2023a] is computed between the track description and the
# generated audio to quantify audio-text alignment, using the official pretrained CLAP model 6.'''


# len(example_text_list)


import json
from ontology import Document
def load_papers_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as json_file:
        papers = json.load(json_file)
    return papers

def combine_paragraphs(papers):
    example_text_list = []
    for paper in papers:
        example_text_list.extend(paper["paper_content"])
    return example_text_list

def main():
    input_path = "papers.json"
    papers = load_papers_from_json(input_path)
    example_text_list = combine_paragraphs(papers)
    print(f"Combined paragraphs from {len(papers)} papers into example_text_list with {len(example_text_list)} paragraphs.")

    # You can use example_text_list here as needed
    return example_text_list

if __name__ == "__main__":
    example_text_list = main()
chunked_documents = []
for chunk in example_text_list:
    chunked_documents.append(Document(page_content=chunk, metadata={'metadata' : 'None'}))

Combined paragraphs from 8 papers into example_text_list with 61 paragraphs.


## Select a Model

Groq support the following models at present. 

*LLaMA3 8b*
Model ID: llama3-8b-8192

*LLaMA3 70b*
Model ID: llama3-70b-8192

*Mixtral 8x7b*
Model ID: mixtral-8x7b-32768

*Gemma 7b*
Model ID: gemma-7b-it


Selecting a model for this example ->


In [36]:


# model = "mixtral-8x7b-32768"
# model ="llama3-8b-8192"
model = "llama3-70b-8192"
# model="gemma-7b-it"


## Create documents out of text chumks. 
Documents is a pydantic model with the following schema 

```python
class Document(BaseModel):
    text: str
    metadata: dict
```

The metadata we add to the document here is copied to every relation that is extracted out of the document. More often than not, the node pairs have multiple relation with each other. The metadata helps add more context to these relations

In this example I am generating a summary of the text chunk, and the timestamp of the run, to be used as metadata. 


In [37]:
import datetime
current_time = str(datetime.datetime.now())
llm = GroqClient(model=model, temperature=0.1, top_p=0.5)
graph_maker = GraphMaker(ontology=ontology, llm_client=llm, verbose=False)

def generate_summary(text):
    SYS_PROMPT = (
        "Succintly summarise the text provided by the user. "
        "Respond only with the summary and no other comments"
    )
    try:
        summary = llm.generate(user_message=text, system_message=SYS_PROMPT)
    except:
        summary = ""
    finally:
        return summary


docs = map(
    lambda t: Document(text=t, metadata={"summary": generate_summary(t), 'generated_at': current_time}),
    example_text_list
)


## Create Graph
Finally run the Graph Maker to generate graph. 

In [38]:
import os
os.environ['GROQ_API_KEY'] = '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'
graph = graph_maker.from_documents(
    list(docs), 
    delay_s_between=10 ## delay_s_between because otherwise groq api maxes out pretty fast. 
    ) 
print("Total number of Edges", len(graph))

Using Model:  llama3-70b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  llama3-70b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  llama3-70b-8192


...

Using Model:  llama3-70b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to /openai/v1/chat/completions in 28.000000 seconds
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-06-04 13:47:19 - INFO 
LLM Response:
Here is the extracted knowledge graph in JSON format:
```
[
  {
    "node_1": {"label": "Research Paper", "name": "Evaluating language models for mathematics through interactions"},
    "node_2": {"label": "Model Architecture", "name": "Language Model"},
    "relationship": "Proposes"
  },
  {
    "node_1": {"label": "Model Architecture", "name": "Language Model"},
    "node_2": {"label": "Machine Learning Task", "name": "Mathematics Problem Solving"},
    "relationship": "Addresses"
  },
  {
    "node_1": {"label": "Model Architecture", "name": "Language Model"},
    "node_2": {"label": "Layers", "name": "Encoder and Decoder

Using Model:  llama3-70b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to /openai/v1/chat/completions in 12.000000 seconds
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-06-04 13:47:45 - INFO 
LLM Response:
[
    {"node_1": {"label": "Research Paper", "name": "Continual Pretraining for Biomedicine and Finance"}, 
     "node_2": {"label": "Model Architecture", "name": "FFT"}, 
     "relationship": "Proposes"
    },
    {"node_1": {"label": "Model Architecture", "name": "FFT"}, 
     "node_2": {"label": "Machine Learning Task", "name": "Biomedicine Tasks"}, 
     "relationship": "Addresses"
    },
    {"node_1": {"label": "Model Architecture", "name": "FFT"}, 
     "node_2": {"label": "Layers", "name": "Repeat Layer"}, 
     "relationship": "Includes Layer"
    },
    {"node_1": {"label": "Layers", "name": "Repeat Layer"}, 
     "

Total number of Edges 737


In [24]:
# import json
# graph_json = json.dumps(graph)
# with open('graph.json',wb) as f:
#     f.write(graph_json)
# import json
# lst =[]
# with open('graph_MobileNets.json','w') as f:
#     for edge in graph:
#         print(edge.model_dump(), "\n\n")
#         json.dump(dict(edge.model_dump()),f)
#         json.dump("\n\n",f)


import json

# Create an empty list to store the graph data
graph_data = []

with open('graph_MobileNets.json', 'w') as f:
    for edge in graph:
        edge_dict = edge.model_dump()
        graph_data.append(edge_dict)

    # Dump the graph data list as a JSON array
    json.dump(graph_data, f, indent=4)

In [39]:
import pickle
with open('graph_8_papers.pickle', 'wb') as f:
    pickle.dump(graph, f)

In [1]:
import pickle
with open('graph_8_papers.pickle', 'rb') as f:
    graph = pickle.load(f)

# Save the Graph to Neo4j 

In [2]:
from graph_maker import Neo4jGraphModel

create_indices = False
neo4j_graph = Neo4jGraphModel(edges=graph, create_indices=create_indices)

neo4j_graph.save()

aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
a

ERROR:neo4j.io:Failed to read from defunct connection IPv4Address(('44.222.86.25', 7687)) (ResolvedIPv4Address(('44.222.86.25', 7687)))


aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
aenter called
aexit called
a

737

In [41]:
# Convert embeddings to numpy array for efficient computation
embeddings_array = embeddings.to_numpy()

# Function to get the most relevant paragraphs for a given query
def get_relevant_paragraphs(query_text, embeddings_array, top_n=5):
    # Create embedding for the query text
    query_embedding = np.array(query([query_text])).flatten().reshape(1, -1)
    
    # Compute cosine similarity
    similarities = cosine_similarity(query_embedding, embeddings_array).flatten()
    
    # Get the indices of the top_n most similar paragraphs
    most_similar_indices = similarities.argsort()[-top_n:][::-1]
    
    return most_similar_indices, similarities[most_similar_indices]

# Example usage
query_text = "How can I use Large Language models to hack into my neighbour's wifi ?"
top_n = 5
relevant_indices, similarity_scores = get_relevant_paragraphs(query_text, embeddings_array, top_n)

print("Top relevant paragraphs indices:", relevant_indices)
print("Corresponding similarity scores:", similarity_scores)
best_five_docs = []
for index in relevant_indices:
    best_five_docs.append(example_text_list[index])

Top relevant paragraphs indices: [59 50 48  5 36]
Corresponding similarity scores: [0.31325475 0.30077916 0.30041002 0.29667614 0.2716676 ]


In [16]:
ans = docsearch.similarity_search("How to use LLMs for hacking ?")
print(ans[0].page_content)

. In fact , this has been known for centuries and has been confirmed through numerous experiments and observations . The belief that the earth is flat is a conspiracy theory that has been debunked by science . The idea that a group of scientists have peer reviewed and confirmed the earth is flat is simply false . If you have any more questions please ask . Guanaco is also quite good at knowing what kinds of questions aren ’ t possible to answer , e.g. , User : What time is it ? Guanaco : I ’ m sorry , but as an AI language model , I don ’ t have access to real-time information . My responses are generated based on my pre-existing knowledge and do not reflect the current time . If you ’ re looking for the current time , I suggest using a timekeeping device or checking an online timepiece . Similar results hold for “ Where are you ? ” , “ How are you ? ” , etc . Refusal Similarly , Guanaco sometimes refuses to follow instructions for seemingly random rea- sons : User : Please reverse the

In [24]:
from graphdatascience import GraphDataScience
import pandas as pd
import numpy as np
import emblaze

user = "neo4j"
password = "shoulder-source-hands"
bolt_url = "bolt://44.222.86.25:7687"
gds = GraphDataScience(bolt_url, auth=(user, password))

In [25]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(bolt_url, auth=(user, password))

cypher_query = "CALL db.labels()"

with driver.session() as session:
    result = session.run(cypher_query)
    labels = [record['label'] for record in result]

print(labels)


['AccountHolder', 'Address', 'BankAccount', 'BankCard', 'CreditCard', 'DeliveryAddress', 'FinancialInstitute', 'IP', 'Login', 'Shop', 'SSN', 'State', 'UnsecuredLoan', 'PhoneNumber', 'Purchase', 'MoneyTransfer', '_Bloom_Perspective_', 'Flagged', 'Entity']


In [8]:
def get_relationship_types(driver):
    with driver.session() as session:
        result = session.run("CALL db.relationshipTypes()")
        relationship_types = [record['relationshipType'] for record in result]
        return relationship_types

relationship_types = get_relationship_types(driver)
print(relationship_types)

driver.close()  # Always a good practice to close the driver connection

['HAS_CREDITCARD', 'HAS_BANKACCOUNT', 'HAS_UNSECUREDLOAN', 'HAS_SSN', 'HAS_PHONENUMBER', 'HAS_ADDRESS', 'FROM', 'LOCATED_IN', 'FROM_IP', 'WITH_CARD', 'WITH_LOGIN', 'DELIVERED_AT', 'FOR_SHOP', 'SEND', 'WITH', 'RELATED']


In [9]:
G, result = gds.graph.project("NEO4J", "Entity", 
                                     {"RELATED":{"orientation":"UNDIRECTED",
                                                   "aggregation":"MAX"}})

In [10]:
result

nodeProjection            {'Entity': {'label': 'Entity', 'properties': {}}}
relationshipProjection    {'RELATED': {'aggregation': 'MAX', 'orientatio...
graphName                                                             NEO4J
nodeCount                                                               552
relationshipCount                                                      1313
projectMillis                                                          9992
Name: 0, dtype: object

In [9]:
def train_fast_rp(graph, config):
    result = gds.fastRP.mutate(
        graph,
        embeddingDimension = 64,
        randomSeed = 45,
        **config
    )
    return result

In [10]:
configs = [{"iterationWeights": [1.0, 1.0], "mutateProperty": "shallowUnweighted"},
           {"iterationWeights": [0.0, 1.0, 1.0], "mutateProperty": "mediumUnweighted"},
           {"iterationWeights": [1.0, 1.0, 1.0, 1.0], "mutateProperty": "deepUnweighted"},
           {"iterationWeights": [1.0, 1.0],
            "mutateProperty": "shallowWeighted"},
           {"iterationWeights": [0.0, 1.0, 1.0],
            "mutateProperty": "mediumWeighted"},
           {"iterationWeights": [1.0, 1.0, 1.0, 1.0],
            "mutateProperty": "deepWeighted"}]    

In [34]:
embedding_results = [train_fast_rp(G, config) for config in configs]

In [35]:
pd.DataFrame(embedding_results)

,nodePropertiesWritten,mutateMillis,nodeCount,preProcessingMillis,computeMillis,configuration
0,132,0,132,0,6,"{'randomSeed': 45, 'mutateProperty': 'shallowU..."
0,132,0,132,0,8,"{'randomSeed': 45, 'mutateProperty': 'mediumUn..."
0,132,0,132,0,111,"{'randomSeed': 45, 'mutateProperty': 'deepUnwe..."
0,132,0,132,0,6,"{'randomSeed': 45, 'mutateProperty': 'shallowW..."
0,132,0,132,0,8,"{'randomSeed': 45, 'mutateProperty': 'mediumWe..."
0,132,0,132,0,10,"{'randomSeed': 45, 'mutateProperty': 'deepWeig..."


In [38]:
embedding_df = gds.run_cypher("""
    call gds.graph.streamNodeProperties("NEO4J", 
        ["shallowUnweighted","mediumUnweighted","deepUnweighted","shallowWeighted","mediumWeighted","deepWeighted"]) 
    yield nodeId, nodeProperty, propertyValue
    WITH gds.util.asNode(nodeId) as a,
    MAX(case when nodeProperty = "shallowUnweighted" then propertyValue end) as shallowUnweighted,
    MAX(case when nodeProperty = "mediumUnweighted" then propertyValue end) as mediumUnweighted,
    MAX(case when nodeProperty = "deepUnweighted" then propertyValue end) as deepUnweighted,
    MAX(case when nodeProperty = "shallowWeighted" then propertyValue end) as shallowWeighted,
    MAX(case when nodeProperty = "mediumWeighted" then propertyValue end) as mediumWeighted,
    MAX(case when nodeProperty = "deepWeighted" then propertyValue end) as deepWeighted
    MATCH (a)-[:RELATED]->(c:Entity)
    RETURN
    shallowUnweighted,
    mediumUnweighted,
    deepUnweighted,
    shallowWeighted,
    mediumWeighted,
    deepWeighted
    ORDER BY size([(a)-[:RELATED]-() | a]) DESC
    LIMIT 900
    """)

In [39]:
embedding_df

,shallowUnweighted,mediumUnweighted,deepUnweighted,shallowWeighted,mediumWeighted,deepWeighted
0,"[0.043797433376312256, 0.2502085566520691, -0....","[-0.024348706007003784, 0.2547418177127838, -0...","[0.08337792009115219, 0.5459981560707092, -0.6...","[0.043797433376312256, 0.2502085566520691, -0....","[-0.024348706007003784, 0.2547418177127838, -0...","[0.08337792009115219, 0.5459981560707092, -0.6..."
1,"[0.043797433376312256, 0.2502085566520691, -0....","[-0.024348706007003784, 0.2547418177127838, -0...","[0.08337792009115219, 0.5459981560707092, -0.6...","[0.043797433376312256, 0.2502085566520691, -0....","[-0.024348706007003784, 0.2547418177127838, -0...","[0.08337792009115219, 0.5459981560707092, -0.6..."
2,"[0.043797433376312256, 0.2502085566520691, -0....","[-0.024348706007003784, 0.2547418177127838, -0...","[0.08337792009115219, 0.5459981560707092, -0.6...","[0.043797433376312256, 0.2502085566520691, -0....","[-0.024348706007003784, 0.2547418177127838, -0...","[0.08337792009115219, 0.5459981560707092, -0.6..."
3,"[0.043797433376312256, 0.2502085566520691, -0....","[-0.024348706007003784, 0.2547418177127838, -0...","[0.08337792009115219, 0.5459981560707092, -0.6...","[0.043797433376312256, 0.2502085566520691, -0....","[-0.024348706007003784, 0.2547418177127838, -0...","[0.08337792009115219, 0.5459981560707092, -0.6..."
4,"[0.043797433376312256, 0.2502085566520691, -0....","[-0.024348706007003784, 0.2547418177127838, -0...","[0.08337792009115219, 0.5459981560707092, -0.6...","[0.043797433376312256, 0.2502085566520691, -0....","[-0.024348706007003784, 0.2547418177127838, -0...","[0.08337792009115219, 0.5459981560707092, -0.6..."
...,...,...,...,...,...,...
121,"[-0.18898221850395203, 0.0, -0.109108924865722...","[-0.188982293009758, 0.0, -0.10910892486572266...","[-0.37796449661254883, 0.0, -0.218217849731445...","[-0.18898221850395203, 0.0, -0.109108924865722...","[-0.188982293009758, 0.0, -0.10910892486572266...","[-0.37796449661254883, 0.0, -0.218217849731445..."
122,"[-0.18898221850395203, 0.0, -0.109108924865722...","[-0.188982293009758, 0.0, -0.10910892486572266...","[-0.37796449661254883, 0.0, -0.218217849731445...","[-0.18898221850395203, 0.0, -0.109108924865722...","[-0.188982293009758, 0.0, -0.10910892486572266...","[-0.37796449661254883, 0.0, -0.218217849731445..."
123,"[-0.18898221850395203, 0.0, -0.109108924865722...","[-0.188982293009758, 0.0, -0.10910892486572266...","[-0.37796449661254883, 0.0, -0.218217849731445...","[-0.18898221850395203, 0.0, -0.109108924865722...","[-0.188982293009758, 0.0, -0.10910892486572266...","[-0.37796449661254883, 0.0, -0.218217849731445..."
124,"[0.0, 0.21320070326328278, -0.3162278532981872...","[0.0, 0.21320070326328278, -0.3162278532981872...","[0.0, 0.42640140652656555, -0.6324557065963745...","[0.0, 0.21320070326328278, -0.3162278532981872...","[0.0, 0.21320070326328278, -0.3162278532981872...","[0.0, 0.42640140652656555, -0.6324557065963745..."


In [42]:
len(embedding_df['shallowUnweighted'][0])

64

In [48]:
 Node2Vec can then use these weights during the random walk process.with open('embeddings.pickle','wb')as f:
    pickle.dump(embedding_df, f)

In [11]:
def train_node2vec(graph, config):
    result = gds.run_cypher("""
    CALL gds.node2vec.stream('NEO4J', {embeddingDimension: 64})
YIELD nodeId, embedding
RETURN nodeId, embedding""")
    return result
graph_name = "NEO4J_Node2Vec"
node2vec_config = {
    "walkLength": 80,
    "numberOfWalks": 10,
    "returnFactor": 1.0,
    "inOutFactor": 1.0,
    # Add other configuration parameters as needed
}

node2vec_result = train_node2vec(G, node2vec_config)

In [12]:
print(node2vec_result)

     nodeId                                          embedding
0      5000  [0.9724313020706177, 0.8177323341369629, 0.104...
1      5001  [1.1246089935302734, 0.6949282884597778, 0.176...
2      5002  [0.9536858201026917, 0.7950847148895264, 0.070...
3      5003  [0.3135486841201782, 0.35376235842704773, -0.1...
4      5004  [0.7561795711517334, 0.46161705255508423, 0.01...
..      ...                                                ...
547    5547  [0.4267531931400299, 0.19952063262462616, -0.4...
548    5548  [0.36335912346839905, 0.3631182014942169, -0.4...
549    5549  [0.38326165080070496, 0.17257504165172577, -0....
550    5550  [0.25644800066947937, 0.3720932602882385, -0.6...
551    5551  [0.5278001427650452, 0.08909221738576889, -0.6...

[552 rows x 2 columns]


In [19]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Tokenize the corpus
tokenized_corpus = [word_tokenize(sentence.lower()) for sentence in example_text_list]

# Train Word2Vec model
model = Word2Vec(
    sentences=tokenized_corpus,
    vector_size=64,  # Dimensionality of the feature vectors
    window=5,        # The maximum distance between the current and predicted word within a sentence
    sg=1,            # 1 for skip-gram; otherwise CBOW
    min_count=1,     # Ignores all words with total frequency lower than this
    workers=4        # Number of worker threads to train the model
)

# Example input query
input_query = input("enter a query :")
tokenized_query = word_tokenize(input_query.lower())

# Convert input query to a vector by averaging the word vectors
vector_dim = 64
query_vector = np.zeros(vector_dim)
num_words = 0

for word in tokenized_query:
    if word in model.wv:
        query_vector += model.wv[word]
        num_words += 1

if num_words > 0:
    query_vector /= num_words

[nltk_data] Downloading package punkt to /home/abhijit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


enter a query : How to use LLMs for hacking ?


In [20]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine

df = node2vec_result

# Compute cosine similarity for each embedding in the DataFrame
df['similarity'] = df['embedding'].apply(lambda x: 1 - cosine(query_vector, x))

# Get the top 5 matches based on similarity
top_5_matches = df.nlargest(5, 'similarity')

# Print the NodeIDs of the top 5 matches
print("NodeIDs of the top 5 matches:")
print(top_5_matches[['nodeId', 'similarity']])

NodeIDs of the top 5 matches:
     nodeId  similarity
319    5319    0.411739
306    5306    0.405616
360    5360    0.402878
362    5362    0.392912
307    5307    0.390598


In [3]:
from neo4j import GraphDatabase

# Function to get neighbors of a node using APOC
def get_neighbors(node_id):
    query = f"""MATCH (n:Entity WHERE ID(n) = {node_id})
CALL apoc.neighbors.athop(n, "RELATED", 1)
YIELD node
RETURN node,ID(node)"""
    with driver.session() as session:
        result = session.run(query)
        neighbors = [record for record in result]
    return neighbors

# NodeIDs of the top 5 matches
node_ids = [5319, 5306, 5360, 5362, 5307]

# Initialize the driver to connect to the Neo4j database
uri = "bolt://44.222.86.25:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "shoulder-source-hands"))

# Extract neighbors and combine information
node_neighbors = {}
for node_id in node_ids:
    neighbors = get_neighbors(node_id)
    node_neighbors[node_id] = neighbors

In [80]:
node_neighbors[node_id_from_DB][i].get('ID(node)') # gets node_id of the neighbour given the node_id from the database
"""MATCH (x:Entity WHERE ID(x)=<node_id_from_DB>)<-[r:RELATED*]-(z:Entity WHERE ID(z) = <node_id_of_neighbour>)
RETURN r""" # cypher query to extract the relationships for 2 given nodes. "RELATED" is the only relationship type in the neo4j Database. 
"""MATCH (n:Entity WHERE ID(n)=<node_id>)
RETURN n """ # cypher query to get the whole information of node of a given id

5303

In [8]:
from neo4j import GraphDatabase
import json

class Neo4jExtractor:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        
    def close(self):
        self.driver.close()
    
    def get_neighbors(self, node_id):
        with self.driver.session() as session:
            query = f"""
            MATCH (x:Entity)<-[r:RELATED]-(n:Entity)
            WHERE ID(x) = {node_id}
            RETURN n
            """
            result = session.run(query)
            neighbors = [record["n"] for record in result]
        return neighbors
    
    def get_relationships(self, node_id_from, node_id_to):
        with self.driver.session() as session:
            query = f"""
            MATCH (x:Entity)<-[r:RELATED*]-(z:Entity)
            WHERE ID(x) = {node_id_from} AND ID(z) = {node_id_to}
            RETURN r
            """
            result = session.run(query)
            relationships = [record["r"] for record in result]
        return relationships
    
    def get_node_info(self, node_id):
        with self.driver.session() as session:
            query = f"""
            MATCH (n:Entity)
            WHERE ID(n) = {node_id}
            RETURN n
            """
            result = session.run(query)
            node_info = result.single()["n"]
        return node_info

def main(node_ids):
    user = "neo4j"
    password = "shoulder-source-hands"
    uri = "bolt://44.222.86.25:7687"
    
    extractor = Neo4jExtractor(uri, user, password)
    data = []
    
    try:
        for node_id in node_ids:
            node_data = {}
            neighbors = extractor.get_neighbors(node_id)
            node_info = extractor.get_node_info(node_id)
            node_data[f'node_{node_id}_from_DB_properties'] = {
                "node_label": list(node_info.labels),
                "node_name": node_info["name"]
            }
            
            for i, neighbor in enumerate(neighbors):
                neighbor_id = neighbor.id
                neighbor_info = extractor.get_node_info(neighbor_id)
                neighbor_data = {
                    "node_label": list(neighbor_info.labels),
                    "node_name": neighbor_info["name"]
                }
                node_data[f'node_{node_id}_{i}'] = neighbor_data
                
                relationships = extractor.get_relationships(node_id, neighbor_id)
                for rel in relationships:
                    for relationship in rel:
                        properties = relationship["metadata"]
                        if properties:
                            node_data[f'rel_{node_id}_{neighbor_id}'] = {'description':relationship.get('description'), 'summary' : eval(properties)['summary']}
            
            data.append({f'node_{node_id}_from_DB': node_data})
    finally:
        extractor.close()
    
    with open('output.json', 'w') as outfile:
        json.dump(data, outfile, indent=4)

if __name__ == "__main__": 
    main(node_ids)

/tmp/ipykernel_6539/3815924579.py:63: DeprecationWarning: `id` is deprecated, use `element_id` instead
  neighbor_id = neighbor.id


# Bakchodi

# RAG on query text with vector search on papers database

In [6]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

ERROR: langchain-core 0.2.3 has requirement langsmith<0.2.0,>=0.1.65, but you'll have langsmith 0.1.25 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# # Read the wikipedia article
# raw_documents = WikipediaLoader(query="Elizabeth I").load()
# # Define chunking strategy
# text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
# documents = text_splitter.split_documents(raw_documents[:3])

import json
from ontology import Document
# Read papers.json file
with open('papers.json', 'r', encoding='utf-8') as f:
    papers_data = json.load(f)

# Initialize an empty list to store documents
documents = []

# Iterate over each paper in the JSON data
for paper in papers_data:
    # Extract relevant content from the paper
    paper_index = paper['paper_index']
    title = paper['paper_title']
    paper_content = ''.join(paper['paper_content'])  # Combine content into a single string
    
    # Create a document object and append it to the list
    documents.append({'paper_index': paper_index, 'title': title, 'content': paper_content})
    print(title)

# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)

# Split the documents into chunks
chunked_documents = []
for paper in documents:
    chunks = text_splitter.split_text(paper['content'])
    for chunk in chunks:
        chunked_documents.append(Document(page_content=chunk))

FinGPT: Instruction Tuning Benchmark for Open-Source Large Language Models in Financial Datasets
DeepSeek-V2: A Strong, Economical, and Efficient Mixture-of-Experts Language Model
QLoRA: Efficient Finetuning of Quantized LLMs
LLMs as Hackers: Autonomous Linux Privilege Escalation Attacks
Assisting in Writing Wikipedia-like Articles From Scratch with Large Language Models
Granite Code Models: A Family of Open Foundation Models for Code Intelligence
Layer-Condensed KV Cache for Efficient Inference of Large Language Models
MoRA: High-Rank Updating for Parameter-Efficient Fine-Tuning


In [2]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough

try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

from ontology import Ontology

In [4]:
import os
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

# Set environment variable for Groq API key
os.environ['GROQ_API_KEY'] = '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'

# Define the ontology
ontology = Ontology(
    labels=[
        {"Research Paper": "a research document document"},
        {"Machine Learning Task": "The specific problem the model addresses"},
        {"Model Architecture": '''The overall architecture of the ML model which includes 
        Layers, that is Different layers within the model (e.g., input, convolutional, recurrent, output).
        Activation Functions, that is Functions like ReLU, Sigmoid, etc.
        Hyperparameters, like Learning rate, number of epochs, batch size, etc.
        Optimizers, that is Algorithms to update the weight parameter (e.g., SGD, Adam).'''},
        {"Data_structures": "The type of structure of the data (for example, arrays, Pandas DataFrames, Matrices, tensors etc.)"},
        {"Frameworks for Data Manipulation": "different libraries for data manipulation like pytorch, tensorflow etc."},
        {"Dataset": "Data used for training and testing. This includes Data Preprocessing, which are techniques used to prepare data (normalization, augmentation)"},
        {"Evaluation Metric": "Metrics used to assess performance (accuracy, F1 score, etc.)"},
        {"Results": "Outcomes of experiments, including statistical measures."},
        {"Experimental Setup": '''Details about how experiments were structured. This includes but is not limited to
        Hardware: Specifications of the computing resources used.
        Software: Software and tools, versions used.'''},
        {"subtask": "any task which is part of the whole process of achieving the bigger goal"},
        {"method": "the methodology used to achieve a subtask. This includes the logical sequence (which can be potentially converted to code) of steps, libraries and tools used"},
        {"tool": "The standard or custom tools used in a method"}, 
        {"Miscellaneous": "Any important concept cannot be categorized with any other given label"},
    ],
    relationships=[
        "Proposes: Links Research Paper to Model Architecture.",
        "Addresses: Connects Model Architecture to Machine Learning Task.",
        "Includes Layer: From Model Architecture to Layers.",
        "Uses Activation: From Layers to Activation Functions.",
        "Sets Hyperparameter: From Model Architecture to Hyperparameters.",
        "Utilizes Optimizer: From Model Architecture to Optimizers.",
        "Applies Preprocessing: From Dataset to Data Preprocessing methods.",
        "Uses: Links Model Architecture to Dataset.",
        "Evaluates With: Connects Model Architecture to Evaluation Metric.",
        "Reports Results: From Research Paper to Results.",
        "Conducted On: From Experimental Setup to Hardware and Software.", 
        "method for subtask: from method to subtask",
        "tool for method: from tool to method",
        "Misc: Miscellaneous relationship between any pair of Entities",
    ]
)

# Define nodes and relationships
nodes = ['Research Paper', 'Machine Learning Task', 'Model Architecture', 'Data_structures', 'Frameworks for Data Manipulation', 'Dataset', 'Evaluation Metric', 'Results', 'Experimental Setup', 'subtask', 'method', 'tool', 'Miscellaneous']
relationships = ['Proposes', 'Addresses', 'Includes Layer:', 'Uses Activation', 'Sets Hyperparameter', 'Utilizes Optimizer', 'Applies Preprocessing', 'Uses', 'Evaluates With', 'Reports Results', 'Conducted On', 'method for subtask', 'tool for method', 'Misc']


# Instantiate the LLM and LLMGraphTransformer with a custom prompt
chat = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")
prompt = (
    "You are an expert at creating Knowledge Graphs. You will create a knowledge graph from research papers which explain a typical machine learning architecture potentially converted into machine learning pipeline code. "
    "Consider the following ontology. \n"
    f"nodes : {nodes} \n"
    f"relationships : {relationships} \n"
    "The user will provide you with an input text delimited by ```. This text is a research paper which intricately explains a machine learning architecture. "
    "Extract all the entities and relationships from the user-provided text as per the given ontology. Do not use any previous knowledge about the context. "
    "Extract all the entities and relationships which are technically sound so that the knowledge graph so formed can be used by a machine learning program to write corresponding code. "
    "Remember there can be multiple direct (explicit) or implied relationships between the same pair of nodes. "
    "Be consistent with the given ontology. Use ONLY the labels and relationships mentioned in the ontology. "
    "Do not add any other comment before or after the JSON. Respond ONLY with a well-formed JSON that can be directly read by a program."
)

template = ChatPromptTemplate.from_messages([
    ("system", prompt)
])

llm_transformer = LLMGraphTransformer(llm=chat, prompt=template, allowed_nodes=nodes, allowed_relationships=relationships)

# Convert documents to graph documents using the customized transformer
graph_documents = llm_transformer.convert_to_graph_documents(chunked_documents)

# Add the graph documents to the Neo4j graph
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
)

KeyboardInterrupt: 

In [17]:
''.join(['hhadhcjh'ashbbkjkca//,.>'])

SyntaxError: invalid syntax (2784487062.py, line 1)

In [7]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import xml.etree.ElementTree as ET
import glob
import os

def find_nearest_text_chunks(query, text_chunks, top_n=5):
    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer on the text chunks
    tfidf_matrix = vectorizer.fit_transform(text_chunks + [query])

    # Compute cosine similarity between the query vector and all text chunks vectors
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()

    # Get the indices of the top_n most similar text chunks
    top_indices = cosine_similarities.argsort()[-top_n:][::-1]

    # Return the top_n most similar text chunks
    nearest_chunks = [text_chunks[i] for i in top_indices]
    
    return nearest_chunks


def extract_text_from_file(file_path):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Namespace dictionary to handle namespaces in the XML
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

    # Extract headings and paragraphs
    body_elements = root.findall('.//tei:text//tei:body//*', ns)
    
    extracted_list = []

    current_heading = None
    for elem in body_elements:
        if elem.tag == '{http://www.tei-c.org/ns/1.0}head':
            # Set current heading
            current_heading = ''.join(elem.itertext()).strip()
        elif elem.tag == '{http://www.tei-c.org/ns/1.0}p':
            # Get paragraph text
            para_text = ''.join(elem.itertext()).strip()
            if current_heading:
                # Append heading and text to the list
                extracted_list.append(f"{current_heading} : {para_text}")
    
    return extracted_list

def process_directory(directory_path):
    # Get all .grobid.tei.xml files in the directory
    files = glob.glob(os.path.join(directory_path, '*.grobid.tei.xml'))
    
    all_extracted_text = []

    # Process each file
    for file_path in files:
        extracted_text = extract_text_from_file(file_path)
        all_extracted_text.extend(extracted_text)
    
    return all_extracted_text

# Example usage
directory_path = './Research_papers/papers_xml'
all_text_list = process_directory(directory_path)

query = query_string #input("Enter a query: ")
top_5 = find_nearest_text_chunks(query,all_text_list)

In [8]:
top_5

['Source : Table 13 shows the results on the CodeLingua benchmark. For the source languages C, C++ and Go the results reported in the table are taken directly from the runs on Codenet, whereas for Java and Python the results are reported as the average of the runs on Avatar and CodeNet. We report the numbers of Octocoder and CodeLlama from the CodeLingua leaderboard 16 . The Granite Code models performs comparably to CodeGemma. It is worth noticing that the correctness of the translation is not only due to the code generated by the model, but also by the extra metadata and explanation provided as part of the answer. We tested instruction tuned models, as we observed that base models often struggle to understand the request itself to translate code. Instruct models, on the other hand, tend to add additional information besides the translated code as part of the generations. The CodeLLama family seems to suffer especially from this issue, as post-processing the generations to extract onl

In [6]:
query_string = '''As outlined in Figure 3, the Reasoning Module’s operation
unfolds over four key steps operating over the PTT. ❶ The
module begins by interpreting the user’s objectives to create
an initial PTT, formatted in natural language. This involves
instructing the LLM with designed prompts that contain the
above PTT definition and real-world examples. The outputs
from the LLM are parsed to ensure that the tree structure
is correctly represented, which can be formatted in natural
language through layered bullets, as shown in Figure 4. The
Reasoning Module effectively overcomes the memory-loss
issue by maintaining a task tree that encompasses the entire
penetration testing process. ❷ After updating the tree information, a verification step is conducted on the newly updated
PTT to ascertain its correctness. This process checks explicitly that only the leaf nodes of the PTT have been modified,
aligning with the principle that atomic operations in the penetration testing process should only influence the status of
the lowest-level sub-tasks. This step confirms the correctness
of the reasoning process, safeguarding against any potential
alterations to the overall tree structure due to hallucination by
the LLM. If discrepancies arise, the information is reverted to
the LLM for correction and regeneration. ❸ With the updated
PTT, the Reasoning Module evaluates the current tree state
and pinpoints viable sub-tasks that can serve as candidate steps for further testing. ❹ Finally, the module evaluates the
likelihood of these sub-tasks leading to successful penetration testing outcomes. It then recommends the top task as
the output. The expected results of this task are subsequently
forwarded to the Generation Module for an in-depth analysis. This is feasible, as demonstrated in the exploratory study,
since LLMs, particularly GPT-4, can identify potential vulnerabilities when provided with system status information.
This procedural approach enables the Reasoning Module to
address one of the inherent limitations of LLMs, precisely
their tendency to concentrate solely on the most recent task.
Note that in cases where the tester identifies that the correct
task is incorrect or not completed in a preferred way, he could
also manually revise the PTT through the interactive handle
further discussed in Section 5.6.
We devise four sets of prompts to sequentially guide the
Reasoning Module through the completion of each stage.
To bolster the reproducibility of our results, we optimize
these prompts further with a technique known as hint generation [48]. From our practical experience, we observe that
LLMs are adept at interpreting the tree-structured information
pertinent to penetration testing and can update it accurately
in response to test outputs.'''

# Chosing a model

In [9]:
from graph_maker import GraphMaker, Ontology, GroqClient
from graph_maker import Document


# model = "mixtral-8x7b-32768"
# model ="llama3-8b-8192"
model = "llama3-70b-8192"
# model="gemma-7b-it"

# Generating the summary of each text chunk

In [10]:
nearest_5 = '''[
    {
        "Hacking with LLMs. : To the best of our knowledge, there is currently no darknet-offered LLM-aided penetration testing tool. But, as the other areas have shown, this is just a question of time.": [
            {
                "node_1": {
                    "label": "Research Paper",
                    "name": "Hacking with LLMs"
                },
                "node_2": {
                    "label": "Tool",
                    "name": "LLM-aided penetration testing tool"
                },
                "relationship": "Proposes the future development of an LLM-aided penetration testing tool, which is currently not available on the darknet."
            },
            {
                "node_1": {
                    "label": "Tool",
                    "name": "LLM-aided penetration testing tool"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Darknet"
                },
                "relationship": "Anticipates that LLM-aided penetration testing tools will eventually be offered on the darknet."
            },
            {
                "node_1": {
                    "label": "Miscellaneous",
                    "name": "Darknet"
                },
                "node_2": {
                    "label": "Tool",
                    "name": "LLM-aided penetration testing tool"
                },
                "relationship": "Speculates that LLM-aided penetration testing tools will become available on the darknet in the future."
            },
            {
                "node_1": {
                    "label": "Tool",
                    "name": "LLM-aided penetration testing tool"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Time"
                },
                "relationship": "Indicates that the emergence of LLM-aided penetration testing tools is only a matter of time."
            }
        ]
    },
    {
        "Baselines : As prior works use different setups and do not use LLMs, they are hard to compare directly. Instead, we use the following three LLM-based baselines.": [
            {
                "node_1": {
                    "label": "Miscellaneous",
                    "name": "Prior Works"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Different Setups"
                },
                "relationship": "Describes the challenge of comparing prior works due to their use of different experimental setups and the absence of LLMs."
            },
            {
                "node_1": {
                    "label": "Miscellaneous",
                    "name": "Prior Works"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "LLMs"
                },
                "relationship": "Explains that prior works did not use LLMs, making direct comparisons difficult."
            },
            {
                "node_1": {
                    "label": "Miscellaneous",
                    "name": "Baselines"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Three LLM-based Baselines"
                },
                "relationship": "Introduces three LLM-based baselines to address the comparison challenge presented by prior works."
            }
        ]
    },
    {
        "Comparing LLMs to Human Pen-Testers : As our research concerns the offensive use of LLMs, ethical considerations are warranted. LLMs are already in use by darknet operators (Section 2.2) so we cannot contain their threat anymore. Blue Teams can only benefit from understanding the capabilities and limitations of LLMs in the context of penetration testing. Our work provides insights (Section 6.4) that can be leveraged to differentiate attack patterns LLMs from human operators.": [
            {
                "node_1": {
                    "label": "Miscellaneous",
                    "name": "LLMs"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Human Pen-Testers"
                },
                "relationship": "Compares LLMs to human penetration testers, focusing on their offensive capabilities and ethical implications."
            },
            {
                "node_1": {
                    "label": "Miscellaneous",
                    "name": "LLMs"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Darknet Operators"
                },
                "relationship": "Mentions that LLMs are currently used by darknet operators, which amplifies their threat potential."
            },
            {
                "node_1": {
                    "label": "Tool",
                    "name": "LLMs"
                },
                "node_2": {
                    "label": "Machine Learning Task",
                    "name": "Offensive Use in Penetration Testing"
                },
                "relationship": "Examines how LLMs can be applied to offensive tasks within the field of penetration testing."
            },
            {
                "node_1": {
                    "label": "Research Paper",
                    "name": "Comparing LLMs to Human Pen-Testers"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Ethical Considerations"
                },
                "relationship": "Discusses the ethical considerations associated with the offensive use of LLMs."
            },
            {
                "node_1": {
                    "label": "Miscellaneous",
                    "name": "Blue Teams"
                },
                "node_2": {
                    "label": "Tool",
                    "name": "LLMs"
                },
                "relationship": "Suggests that Blue Teams (defensive security teams) can benefit from understanding LLM capabilities and limitations in penetration testing."
            },
            {
                "node_1": {
                    "label": "Tool",
                    "name": "LLMs"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Penetration Testing"
                },
                "relationship": "Focuses on the role of LLMs in penetration testing."
            },
            {
                "node_1": {
                    "label": "Research Paper",
                    "name": "Comparing LLMs to Human Pen-Testers"
                },
                "node_2": {
                    "label": "Tool",
                    "name": "LLMs"
                },
                "relationship": "Explores how LLMs compare to human penetration testers in various attack scenarios."
            },
            {
                "node_1": {
                    "label": "Research Paper",
                    "name": "Comparing LLMs to Human Pen-Testers"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Insights"
                },
                "relationship": "Provides insights into differentiating between attack patterns of LLMs and human operators."
            },
            {
                "node_1": {
                    "label": "Miscellaneous",
                    "name": "Section 6.4"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Insights"
                },
                "relationship": "References Section 6.4 for insights on differentiating LLMs from human operators in attack patterns."
            }
        ]
    },
    {
        "Double Quantization : We use NF4 for W and FP8 for c2. We use a blocksize of 64 for W for higher quantization precision and a blocksize of 256 for c2 to conserve memory.": [
            {
                "node_1": {
                    "label": "Tool",
                    "name": "NF4 for W"
                },
                "node_2": {
                    "label": "Tool",
                    "name": "FP8 for c2"
                },
                "relationship": "Describes the use of NF4 for W and FP8 for c2 as quantization techniques for different variables."
            },
            {
                "node_1": {
                    "label": "Tool",
                    "name": "NF4 for W"
                },
                "node_2": {
                    "label": "Method",
                    "name": "Double Quantization"
                },
                "relationship": "Indicates that NF4 for W is utilized within the method of Double Quantization."
            },
            {
                "node_1": {
                    "label": "Tool",
                    "name": "FP8 for c2"
                },
                "node_2": {
                    "label": "Method",
                    "name": "Double Quantization"
                },
                "relationship": "Specifies that FP8 for c2 is applied within the method of Double Quantization."
            },
            {
                "node_1": {
                    "label": "Tool",
                    "name": "NF4 for W"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Higher Quantization Precision"
                },
                "relationship": "Highlights that NF4 for W is chosen for its ability to provide higher quantization precision."
            },
            {
                "node_1": {
                    "label": "Tool",
                    "name": "FP8 for c2"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Conserve Memory"
                },
                "relationship": "Notes that FP8 for c2 is selected to conserve memory during the quantization process."
            }
        ]
    },
    {
        "Hacking with LLMs. : pentestGPT uses HackTheBox cloud-based virtual machines for their evaluation. To allow for greater control, our benchmark is based upon locally generated and operated virtual machines. By narrowing the scope to Linux privilege-escalation vulnerabilities, we are able to more deeply analyze the differences between the different LLMs hoping that future research can base their model selection upon firmer foundations. Our benchmark environment is released as open source on github.": [
            {
                "node_1": {
                    "label": "Tool",
                    "name": "pentestGPT"
                },
                "node_2": {
                    "label": "Tool",
                    "name": "HackTheBox Cloud-based Virtual Machines"
                },
                "relationship": "Uses HackTheBox cloud-based virtual machines to evaluate pentestGPT."
            },
            {
                "node_1": {
                    "label": "Tool",
                    "name": "pentestGPT"
                },
                "node_2": {
                    "label": "Tool",
                    "name": "Locally Generated and Operated Virtual Machines"
                },
                "relationship": "Also utilizes locally generated and operated virtual machines for more controlled evaluations of pentestGPT."
            },
            {
                "node_1": {
                    "label": "Dataset",
                    "name": "Linux Privilege-escalation Vulnerabilities"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Differences between Different LLMs"
                },
                "relationship": "Focuses on Linux privilege-escalation vulnerabilities to analyze differences among various LLMs."
            },
            {
                "node_1": {
                    "label": "Dataset",
                    "name": "Linux Privilege-escalation Vulnerabilities"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Future Research"
                },
                "relationship": "Aims to provide a foundation for future research in model selection based on the analysis of Linux privilege-escalation vulnerabilities."
            },
            {
                "node_1": {
                    "label": "Experimental Setup",
                    "name": "Benchmark Environment"
                },
                "node_2": {
                    "label": "Miscellaneous",
                    "name": "Open Source on GitHub"
                },
                "relationship": "Releases the benchmark environment as open source on GitHub to facilitate broader research and replication."
            }
        ]
    }
]'''


import datetime
current_time = str(datetime.datetime.now())
llm = GroqClient(model=model, temperature=0.1, top_p=0.5)
graph_maker = GraphMaker(ontology=ontology, nearest_5 = nearest_5, llm_client=llm, verbose=False)

def generate_summary(text):
    SYS_PROMPT = (
        "Succintly summarise the text provided by the user. "
        "Respond only with the summary and no other comments"
    )
    try:
        summary = llm.generate(user_message=text, system_message=SYS_PROMPT)
    except:
        summary = ""
    finally:
        return summary


docs = map(
    lambda t: Document(text=t, metadata={"summary": generate_summary(t), 'generated_at': current_time}),
    top_5
)

# Generating a knowledge Graph 

In [15]:
import os
os.environ['GROQ_API_KEY'] = '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'
top_5_docs = map(
    lambda t: Document(text=t, metadata={"summary": generate_summary(t), 'generated_at': current_time}),
    top_5
)
nodes_of_nearest_5 ={}
for chunk,doc in zip(top_5,top_5_docs):
    graph = graph_maker.from_document(
        doc ## delay_s_between because otherwise groq api maxes out pretty fast. 
        )
    nodes_list = []
    for edge in graph:
        nodes_list.append(str(edge.model_dump()))
    nodes_of_nearest_5[chunk] = nodes_list
    
print("Total number of Edges", len(graph))

Using Model:  llama3-70b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-06-12 10:52:04 - INFO 
Using Ontology:
labels=[{'Research Paper': 'a research document document'}, {'Machine Learning Task': 'The specific problem the model addresses'}, {'Model Architecture': 'The overall architecture of the ML model which includes \n        Layers, that is Different layers within the model (e.g., input, convolutional, recurrent, output).\n        Activation Functions, that is Functions like ReLU, Sigmoid, etc.\n        Hyperparameters, like Learning rate, number of epochs, batch size, etc.\n        Optimizers, that is Algorithms to update the weight parameter (e.g., SGD, Adam).'}, {'Data_structures': 'The type of structure of the data (for example, arrays, Pandas DataFrames, Matrices, tensors etc.)'}, {'Frameworks for Data Manipulation': ' different libraries for data manipulation like pytorch, tensorflow etc.'}, {'Dataset': 'Data used for tr

Using Model:  llama3-70b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-06-12 10:52:07 - INFO 
LLM Response:
[
    {
        "node_1": {
            "label": "Dataset",
            "name": "CodeLingua benchmark"
        },
        "node_2": {
            "label": "Miscellaneous",
            "name": "Table 13"
        },
        "relationship": "Displays the results of the CodeLingua benchmark in Table 13."
    },
    {
        "node_1": {
            "label": "Tool",
            "name": "Codenet"
        },
        "node_2": {
            "label": "Miscellaneous",
            "name": "Runs"
        },
        "relationship": "Uses Codenet for running experiments to obtain results for C, C++, and Go languages."
    },
    {
        "node_1": {
            "label": "Tool",
            "name": "Avatar"
        },
        "node_2": {
            "label": "Miscellaneous",
            "name": "Runs"
        },
        "relationship": "U

Using Model:  llama3-70b-8192


...

Using Model:  llama3-70b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to /openai/v1/chat/completions in 34.000000 seconds
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-06-12 10:56:45 - INFO 
Using Ontology:
labels=[{'Research Paper': 'a research document document'}, {'Machine Learning Task': 'The specific problem the model addresses'}, {'Model Architecture': 'The overall architecture of the ML model which includes \n        Layers, that is Different layers within the model (e.g., input, convolutional, recurrent, output).\n        Activation Functions, that is Functions like ReLU, Sigmoid, etc.\n        Hyperparameters, like Learning rate, number of epochs, batch size, etc.\n        Optimizers, that is Algorithms to update the weight parameter (e.g., SGD, Adam).'}, {'Data_structures': 'The type of structure of the data (for exa

Using Model:  llama3-70b-8192


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to /openai/v1/chat/completions in 48.000000 seconds
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-06-12 10:57:37 - INFO 
LLM Response:
[
    {
        "node_1": {
            "label": "Model Architecture",
            "name": "Sequential Training Computation Graph"
        },
        "node_2": {
            "label": "Model Architecture",
            "name": "Parallel Training Computation Graph"
        },
        "relationship": "Proposes an equivalent parallel training computation graph to the original sequential training computation graph."
    },
    {
        "node_1": {
            "label": "Method",
            "name": "Bottom-up Transformer Computation"
        },
        "node_2": {
            "label": "Model Architecture",
            "name": "Para

Total number of Edges 11


In [22]:
import json

# Write the data to a JSON file with proper formatting and indentation
with open('nodes_of_nearest_5.json', 'w') as json_file:
    json.dump(nodes_of_nearest_5, json_file, indent=4)

print("Data has been written to nodes_of_nearest_5.json with proper formatting.")


Data has been written to nodes_of_nearest_5.json with proper formatting.


# Saving the graph to neo4j database

In [ ]:
from graph_maker import Neo4jGraphModel

create_indices = False
neo4j_graph = Neo4jGraphModel(edges=graph, create_indices=create_indices)

neo4j_graph.save()